In [ ]:
import os
import torch
import torch_xla.core.xla_model as xm
from transformers import AutoTokenizer, AutoModelForCausalLM

print("Étape 0: Initialisation.")

os.environ["XLA_USE_BF16"] = "1"
os.environ["TORCH_XLA_DEBUG"] = "0"
MODEL_ID = "utter-project/EuroLLM-1.7B"

# --- ÉTAPE 1 ---
print("\nÉtape 1: Connexion au TPU...")
device = xm.xla_device()
print(f"Device TPU trouvé: {device}")

# --- ÉTAPE 2 ---
print("\nÉtape 2: Téléchargement du tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
print("Tokenizer téléchargé.")

# --- ÉTAPE 3 ---
print("\nÉtape 3: Téléchargement du modèle...")
model = AutoModelForCausalLM.from_pretrained(MODEL_ID, torch_dtype=torch.bfloat16)
model.config.use_cache = False  # XLA incompatible
model.to(device)
xm.mark_step()
print("Modèle prêt sur TPU.")

# --- ÉTAPE 4 ---
prompt = "Hello, how are you today"
print("\nÉtape 4: Tokenisation du prompt...")
inputs = tokenizer(prompt, return_tensors="pt").to(device)
input_ids = inputs["input_ids"]
print("Prompt tokenizé.")

# --- ÉTAPE 5 ---
print("\nÉtape 5: Warm-up TPU (compilation initiale courte)...")
with torch.no_grad():
    _ = model(input_ids)
xm.mark_step()
print("Warm-up terminé.")

# --- ÉTAPE 6 ---
print("\nÉtape 6: Génération manuelle token par token (optimisée pour TPU)...")

max_new_tokens = 64
generated_ids = input_ids
model.eval()

with torch.no_grad():
    for i in range(max_new_tokens):
        # Forward
        outputs = model(generated_ids)
        next_token_logits = outputs.logits[:, -1, :]

        # Prendre le token le plus probable (greedy)
        next_token = torch.argmax(next_token_logits, dim=-1, keepdim=True)

        # Ajouter le nouveau token à la séquence
        generated_ids = torch.cat([generated_ids, next_token], dim=-1)

        # Synchronisation TPU
        xm.mark_step()

        # Vérifie si fin de séquence atteinte
        if next_token.item() == tokenizer.eos_token_id:
            print(f"\nArrêt précoce à {i+1} tokens (EOS atteint).")
            break

        # Affiche un aperçu tous les 10 tokens
        if (i + 1) % 10 == 0:
            partial = tokenizer.decode(generated_ids[0].cpu(), skip_special_tokens=True)
            print(f"→ {i+1} tokens générés...")
            print(partial)
            print("-" * 50)

print("\nGénération terminée.")

# --- ÉTAPE 7 ---
print("\nÉtape 7: Décodage final...")
generated_text = tokenizer.decode(generated_ids[0].cpu(), skip_special_tokens=True)

print("\n--- RÉSULTAT FINAL ---")
print(generated_text)

Étape 0: Initialisation.

Étape 1: Connexion au TPU...
Device TPU trouvé: xla:0

Étape 2: Téléchargement du tokenizer...


/tmp/ipython-input-1141579989.py:14: DeprecationWarning: Use torch_xla.device instead
  device = xm.xla_device()


Tokenizer téléchargé.

Étape 3: Téléchargement du modèle...


/tmp/ipython-input-1141579989.py:29: DeprecationWarning: Use torch_xla.sync instead
  xm.mark_step()
/tmp/ipython-input-1141579989.py:43: DeprecationWarning: Use torch_xla.sync instead
  xm.mark_step()


Modèle prêt sur TPU.

Étape 4: Tokenisation du prompt...
Prompt tokenizé.

Étape 5: Warm-up TPU (compilation initiale courte)...
Warm-up terminé.

Étape 6: Génération manuelle token par token (optimisée pour TPU)...


/tmp/ipython-input-1141579989.py:66: DeprecationWarning: Use torch_xla.sync instead
  xm.mark_step()


→ 10 tokens générés...
Hello, how are you today?
I hope you are doing well.

--------------------------------------------------
→ 20 tokens générés...
Hello, how are you today?
I hope you are doing well.
I am writing to you today to let you know
--------------------------------------------------
→ 30 tokens générés...
Hello, how are you today?
I hope you are doing well.
I am writing to you today to let you know that I have been awarded a scholarship to study at
--------------------------------------------------
→ 40 tokens générés...
Hello, how are you today?
I hope you are doing well.
I am writing to you today to let you know that I have been awarded a scholarship to study at the University of Oxford.
I am very excited
--------------------------------------------------
→ 50 tokens générés...
Hello, how are you today?
I hope you are doing well.
I am writing to you today to let you know that I have been awarded a scholarship to study at the University of Oxford.
I am very excited about 